# Tokenizer Class

> Implementation of the `Tokenizer` class, which encapsulates the BPE vocabulary and merge rules to provide encoding and decoding functionalities.

In [ ]:
#| default_exp bpe_tokenizer_class

In [9]:
#| hide
from nbdev.showdoc import *

In [10]:
#| export
import sys
from pathlib import Path
import os

# --- Project Root Setup --- 
project_root = Path(os.getcwd())
# Check if running from nbs/
if project_root.name == 'nbs' and (project_root.parent / 'settings.ini').exists():
    project_root = project_root.parent
elif not (project_root / 'settings.ini').exists():
     # Try going up one level if settings.ini not found directly
     if (project_root.parent / 'settings.ini').exists():
          project_root = project_root.parent
     else:
          print("Warning: Could not automatically determine project root based on 'settings.ini'. Assuming current dir or its parent might be it, or sys.path is already correct.")
          # Fallback: Assume running from project root if structure unknown

project_root_str = str(project_root.resolve())
if project_root_str not in sys.path:
    # print(f"Adding project root to sys.path: {project_root_str}") # Verbose option
    sys.path.insert(0, project_root_str)
else:
    # print(f"Project root already in sys.path: {project_root_str}") # Less verbose
    pass
# --- End Project Root Setup --- 

In [11]:
#| export
import regex as re
import json
import base64
from typing import List, Dict, Tuple, Optional, Iterable, Iterator, Any, Pattern
import os # For temp file creation in example
import tempfile # For temp file creation in example

# Attempt to import from the installed/exported library first
try:
    from lm.utils import load_tokenizer_components, save_tokenizer_components
except ImportError:
    # Fallback for direct notebook execution if 'bpe_tokenizer_scratch' isn't in sys.path yet
    # (sys.path modification already attempted in the cell above)
    try:
        from lm.utils import load_tokenizer_components, save_tokenizer_components
        # print("Successfully imported from bpe_tokenizer_scratch.utils after sys.path check/modification.")
    except ImportError:
        # This is the final fallback if the import still fails.
        print("Warning: 'bpe_tokenizer_scratch.utils' module not found even after attempting sys.path modification. \n"
              "Defining load/save_tokenizer_components locally. This might happen if: \n"
              "  1. '00_utils.ipynb' has not been exported via `nbdev_export`.\n"
              "  2. The project structure differs from the assumed layout (e.g., not in 'nbs/' subdirectory). \n"
              "  3. The library name in 'settings.ini' (lib_name) is not 'bpe_tokenizer_scratch'.\n"
              "Ensure the package is correctly structured and exported, or install it via 'pip install -e .'.")
        
        # Fallback: Define directly if utils not available
        def save_tokenizer_components(vocab: Dict[int, bytes], merges: List[Tuple[bytes, bytes]], vocab_filepath: str, merges_filepath: str):
            vocab_to_save = {str(k): base64.b64encode(v).decode('ascii') for k, v in vocab.items()}
            with open(vocab_filepath, 'w', encoding='utf-8') as f: json.dump(vocab_to_save, f, indent=2)
            merges_to_save = [(base64.b64encode(p1).decode('ascii'), base64.b64encode(p2).decode('ascii')) for p1, p2 in merges]
            with open(merges_filepath, 'w', encoding='utf-8') as f: json.dump(merges_to_save, f, indent=2)

        def load_tokenizer_components(vocab_filepath: str, merges_filepath: str) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
            with open(vocab_filepath, 'r', encoding='utf-8') as f: vocab_json = json.load(f)
            loaded_vocab = {int(k): base64.b64decode(v_str) for k, v_str in vocab_json.items()}
            with open(merges_filepath, 'r', encoding='utf-8') as f: merges_json = json.load(f)
            loaded_merges = [(base64.b64decode(p1_str), base64.b64decode(p2_str)) for p1_str, p2_str in merges_json]
            return loaded_vocab, loaded_merges

In [12]:
#| export
class Tokenizer:
    """ A BPE Tokenizer that can encode text into token IDs and decode token IDs back to text.
    
    The Tokenizer is initialized with a vocabulary (mapping IDs to byte sequences) 
    and a list of merge rules (ordered pairs of byte sequences that were merged during training).
    It also handles special tokens.
    """
    PAT_REGEX: Pattern = re.compile(r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
    
    def __init__(self, 
                 vocab: Dict[int, bytes], 
                 merges: List[Tuple[bytes, bytes]], 
                 special_tokens: Optional[List[str]] = None):
        """Initializes the Tokenizer.

        Args:
            vocab (Dict[int, bytes]): A dictionary mapping token IDs (int) to their byte representations (bytes).
                                     This vocab should ideally come from `train_bpe` and include base bytes (0-255)
                                     and any initial special tokens passed to `train_bpe`.
            merges (List[Tuple[bytes, bytes]]): An ordered list of BPE merge rules. Each rule is a tuple of two
                                                byte sequences that were merged. The order determines merge priority.
            special_tokens (Optional[List[str]]): A list of special token strings (e.g., "<|endoftext|>"). 
                                                    These tokens are handled distinctly during encoding/decoding.
                                                    If a special token's byte representation is already in `vocab`,
                                                    its existing ID is used. Otherwise, a new ID is assigned and 
                                                    it's added to the vocabulary.
        """
        self.vocab: Dict[int, bytes] = vocab.copy() 
        self.merges_map: Dict[Tuple[bytes, bytes], int] = {pair: i for i, pair in enumerate(merges)} 

        self.special_tokens_list: List[str] = []
        if special_tokens is not None:
            seen_st = set()
            for st in special_tokens:
                if st not in seen_st:
                    self.special_tokens_list.append(st)
                    seen_st.add(st)
            
        self.special_token_to_id: Dict[str, int] = {} 
        self.id_to_special_token: Dict[int, str] = {} 

        _temp_bytes_to_id: Dict[bytes, int] = {b: i for i, b in sorted(self.vocab.items(), key=lambda item: item[0])}
        
        max_internal_id = -1
        if self.vocab: 
            max_internal_id = max((k for k in self.vocab.keys() if isinstance(k, int)), default=-1)
        
        current_max_id_for_new_specials = max_internal_id

        for st_text in self.special_tokens_list:
            st_bytes = st_text.encode("utf-8")
            if st_bytes in _temp_bytes_to_id:
                existing_id = _temp_bytes_to_id[st_bytes]
                self.special_token_to_id[st_text] = existing_id
                self.id_to_special_token[existing_id] = st_text
                self.vocab[existing_id] = st_bytes 
            else:
                current_max_id_for_new_specials += 1
                new_id = current_max_id_for_new_specials
                
                self.vocab[new_id] = st_bytes      
                _temp_bytes_to_id[st_bytes] = new_id 
                
                self.special_token_to_id[st_text] = new_id
                self.id_to_special_token[new_id] = st_text
        
        self.bytes_to_id: Dict[bytes, int] = {b: i for i, b in self.vocab.items()}
        
        if self.special_tokens_list:
            sorted_unique_special_tokens_for_regex = sorted(list(set(self.special_tokens_list)), key=len, reverse=True)
            self.special_tokens_regex_pattern = f"({'|'.join(map(re.escape, sorted_unique_special_tokens_for_regex))})"
            self.special_tokens_regex: Optional[Pattern] = re.compile(self.special_tokens_regex_pattern)
        else:
            self.special_tokens_regex: Optional[Pattern] = None
    
    @classmethod
    def from_files(cls, 
                   vocab_filepath: str, 
                   merges_filepath: str, 
                   special_tokens: Optional[List[str]] = None) -> 'Tokenizer':
        """Loads a Tokenizer from vocabulary and merges files.

        Args:
            vocab_filepath (str): Path to the JSON file containing the vocabulary.
            merges_filepath (str): Path to the JSON file containing the merge rules.
            special_tokens (Optional[List[str]]): A list of special token strings.

        Returns:
            Tokenizer: An instance of the Tokenizer class initialized with the loaded data.
        
        Raises:
            FileNotFoundError: If vocab_filepath or merges_filepath does not exist.
            json.JSONDecodeError: If the files are not valid JSON.
            TypeError: If the loaded data is not in the expected format.
        """
        loaded_vocab: Dict[int, bytes]
        loaded_merges: List[Tuple[bytes, bytes]]
        
        if 'load_tokenizer_components' in globals() and callable(globals()['load_tokenizer_components']):
            try:
                 loaded_vocab, loaded_merges = globals()['load_tokenizer_components'](vocab_filepath, merges_filepath)
            except Exception as e:
                print(f"Error using 'load_tokenizer_components' (potentially from utils.py): {e}")
                raise
        else:
            print("Critical: 'load_tokenizer_components' is not defined or not callable. Cannot load from files.")
            raise NameError("Helper function 'load_tokenizer_components' not available.")
        
        return cls(loaded_vocab, loaded_merges, special_tokens)

    def _encode_bytes_with_merges(self, piece_bytes: bytes) -> List[int]:
        """Encodes a piece of bytes (typically a pre-tokenized word) using BPE merges.

        Args:
            piece_bytes (bytes): The byte sequence of a pre-tokenized word.

        Returns:
            List[int]: A list of token IDs representing the encoded piece.
        """
        if not piece_bytes: 
            return []
        
        tokens_as_bytes_list: List[bytes] = [bytes([b]) for b in piece_bytes]

        while len(tokens_as_bytes_list) > 1:
            best_pair_info: Optional[Tuple[int, int]] = None 
            
            for i in range(len(tokens_as_bytes_list) - 1):
                pair = (tokens_as_bytes_list[i], tokens_as_bytes_list[i+1])
                if pair in self.merges_map:
                    rank = self.merges_map[pair]
                    if best_pair_info is None or rank < best_pair_info[0]:
                        best_pair_info = (rank, i)
            
            if best_pair_info is None:
                break 

            _rank, idx = best_pair_info 
            merged_token_bytes = tokens_as_bytes_list[idx] + tokens_as_bytes_list[idx+1]
            tokens_as_bytes_list = tokens_as_bytes_list[:idx] + [merged_token_bytes] + tokens_as_bytes_list[idx+2:]
        
        final_ids: List[int] = []
        for b_sequence in tokens_as_bytes_list:
            if b_sequence in self.bytes_to_id:
                final_ids.append(self.bytes_to_id[b_sequence])
            else:
                for single_byte_val in b_sequence: 
                    single_byte_obj = bytes([single_byte_val])
                    if single_byte_obj in self.bytes_to_id:
                         final_ids.append(self.bytes_to_id[single_byte_obj])
                    else:
                        print(f"Critical Warning: Single byte {single_byte_obj!r} (value: {single_byte_val}) not in bytes_to_id map. Vocab may be malformed or incomplete.")
                        pass 
        return final_ids
    
    def encode(self, text: str) -> List[int]:
        """Encodes a string of text into a list of token IDs.

        The encoding process involves:
        1. Splitting the text by special tokens (if any are defined and present).
        2. For segments that are special tokens, their pre-assigned IDs are used.
        3. For other segments, GPT-2's pre-tokenization regex (PAT_REGEX) is applied to get "words".
        4. Each "word" is converted to UTF-8 bytes.
        5. BPE merges are applied to these bytes via `_encode_bytes_with_merges`.
        6. All resulting token IDs are concatenated.

        Args:
            text (str): The input string to encode.

        Returns:
            List[int]: A list of token IDs representing the encoded text.
        """
        token_ids: List[int] = []
        
        if not text: # Handle empty string input immediately
            return []

        segments_to_process: List[str]
        if not self.special_tokens_regex: 
            segments_to_process = [text]
        else:
            segments_to_process = self.special_tokens_regex.split(text)

        for segment in segments_to_process:
            if not segment: 
                continue

            if segment in self.special_token_to_id:
                token_ids.append(self.special_token_to_id[segment])
            else:
                for pre_token_match in self.PAT_REGEX.finditer(segment):
                    pre_token_str = pre_token_match.group(0)
                    pre_token_bytes = pre_token_str.encode("utf-8") 
                    token_ids.extend(self._encode_bytes_with_merges(pre_token_bytes))
        return token_ids

    def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]:
        """Encodes an iterable of text strings, yielding token IDs.

        Args:
            iterable (Iterable[str]): An iterable (e.g., a list or generator) of strings to encode.

        Yields:
            int: Token IDs from the encoded text chunks.
        """
        for text_chunk in iterable:
            for token_id in self.encode(text_chunk):
                yield token_id
                
    def decode(self, ids: List[int]) -> str:
        """Decodes a list of token IDs back into a string.

        Args:
            ids (List[int]): A list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        byte_chunks: List[bytes] = []
        for token_id in ids:
            if token_id in self.id_to_special_token:
                byte_chunks.append(self.id_to_special_token[token_id].encode('utf-8'))
            elif token_id in self.vocab:
                # Ensure that self.vocab[token_id] is indeed bytes
                token_val = self.vocab[token_id]
                if isinstance(token_val, bytes):
                    byte_chunks.append(token_val)
                else:
                    # This case should not happen if vocab is constructed correctly
                    print(f"Warning: Vocab item for ID {token_id} is not bytes: {token_val!r}. Skipping.")
                    pass 
            else:
                # print(f"Warning: Unknown token ID {token_id} encountered during decode. Skipping.")
                pass 
        
        full_byte_sequence = b"".join(byte_chunks)
        return full_byte_sequence.decode("utf-8", errors="replace")

In [13]:
#| hide
# Example to test methods including encode, encode_iterable, and decode
if __name__ == '__main__':
    print("--- Setting up Tokenizer for tests ---")

    # Corrected vocab_for_tests: Values must be bytes.
    # IDs 0-255 map to their respective single byte representations.
    # Merged tokens use IDs >= 256.
    vocab_for_tests = {
        **{i: bytes([i]) for i in range(256)}, 
        # Merged token IDs and their byte sequences:
        256: b'\xe2\x82\xac', # Euro sign: €
        257: b'ell',         # Represents the merge of 'e' and 'll' (hypothetically)
        258: b'ello',        # Represents the merge of 'ell' and 'o'
        259: b' wo',         # Represents ' ' and 'w' merged, then with 'o'
        260: b' wor',        # Represents ' wo' and 'r'
        261: b'orld',        # Represents 'orl' and 'd'
        262: b' world'       # Represents ' wor' and 'ld'
    }

    merges_for_tests = [
        (b'e', b'l'),      # rank 0
        (b'el', b'l'),     # rank 1 -> b'ell' (ID 257)
        (b'ell', b'o'),    # rank 2 -> b'ello' (ID 258)
        (b' ', b'w'),      # rank 3
        (b' w', b'o'),     # rank 4 -> b' wo' (ID 259)
        (b' wo', b'r'),    # rank 5 -> b' wor' (ID 260)
        (b'o', b'r'),      # rank 6
        (b'or', b'l'),     # rank 7
        (b'orl', b'd'),    # rank 8 -> b'orld' (ID 261)
        # Example: (b' wor', b'ld') -> b' world' (ID 262) is rank 9 (if b'ld' is formed first)
        # For simplicity of testing, let's assume the merges directly result in the tokens in vocab.
        # The example vocab implies b'orld' is a token, and b' world' is also a token.
        # Let's add a merge that could form b' world' if b'ld' was also a merged token
        (b' wor', bytes([108, 100])) # Example for (b' wor', b'ld') -> Not used if b'ld' isn't a token itself
    ]
    # Ensure merges_for_tests only contains pairs that would form tokens present in vocab_for_tests or lead to them.
    # For this test, we care more that the merges_map is built correctly and applied.

    special_tokens_for_tests = ["<EOS>", "<PAD>"]
    
    tokenizer_for_tests = Tokenizer(vocab_for_tests.copy(), merges_for_tests, special_tokens_for_tests)
    
    eos_id = tokenizer_for_tests.special_token_to_id["<EOS>"] # Should be 263
    pad_id = tokenizer_for_tests.special_token_to_id["<PAD>"] # Should be 264
    print(f"Assigned <EOS> ID: {eos_id}, <PAD> ID: {pad_id}")

    print("\n--- Testing decode method ---")
    decode_test_cases = {
        "simple sequence": ([ord('h'), 258, 262, ord('!')], "hello world!"),
        "sequence with special token": ([ord('h'), 258, eos_id, 262], "hello<EOS> world"),
        "empty list": ([], ""),
        "unknown token ID": ([ord('h'), 258, 999, 262], "hello world"), 
        "only special tokens": ([eos_id, pad_id, eos_id], "<EOS><PAD><EOS>"),
        "unicode character from vocab": ([256], "€")
    }
    for name, (input_ids, expected_text) in decode_test_cases.items():
        try:
            result_text = tokenizer_for_tests.decode(input_ids)
            assert result_text == expected_text, f"Decode Test '{name}' FAILED. Expected '{expected_text}', Got '{result_text}' for IDs {input_ids}"
            print(f"Decode Test '{name}' PASSED.")
        except Exception as e:
            print(f"Decode Test '{name}' ERRORED for IDs {input_ids}: {e}")
            import traceback; traceback.print_exc()

    print("\n--- Testing encode/decode roundtrip ---")
    roundtrip_test_texts = [
        "hello world!", 
        "<EOS>hello<PAD> world€!", 
        "复杂的中文字符串", 
        "",
        "   leading and trailing spaces   ",
        "'s 'd 'll 've 're 'm 't",
        "A single byte like z then a special <EOS> token."
    ]
    for i, text in enumerate(roundtrip_test_texts):
        try:
            encoded_ids = tokenizer_for_tests.encode(text)
            decoded_text = tokenizer_for_tests.decode(encoded_ids)
            assert decoded_text == text, f"Roundtrip Test {i} FAILED for '{text}'. Encoded: {encoded_ids}, Decoded: '{decoded_text}'"
            print(f"Roundtrip Test {i} for '{text[:50]}' PASSED.") # Truncate long text for printing
        except Exception as e:
            print(f"Roundtrip Test {i} for '{text[:50]}' ERRORED: {e}")
            import traceback; traceback.print_exc()

    print("\n--- Testing decode with malformed UTF-8 ---")
    temp_vocab_malformed = tokenizer_for_tests.vocab.copy()
    malformed_id = 1000 
    temp_vocab_malformed[malformed_id] = b'\xff\xfe' # Invalid UTF-8 sequence
    tokenizer_malformed_test = Tokenizer(temp_vocab_malformed, merges_for_tests, special_tokens_for_tests)
    
    ids_with_malformed = [ord('h'), malformed_id, ord('i')]
    expected_malformed_decode = "h\ufffd\ufffdi" 
    decoded_malformed_text = tokenizer_malformed_test.decode(ids_with_malformed)
    assert decoded_malformed_text == expected_malformed_decode, \
        f"Decode malformed UTF-8 FAILED. Expected '{expected_malformed_decode}', Got '{decoded_malformed_text}'"
    print(f"Decode malformed UTF-8 PASSED. Decoded: '{decoded_malformed_text!r}'")

    print("\nAll Tokenizer class method tests (init, from_files, _encode_bytes, encode, encode_iterable, decode) completed.")


--- Setting up Tokenizer for tests ---
Assigned <EOS> ID: 263, <PAD> ID: 264

--- Testing decode method ---
Decode Test 'simple sequence' PASSED.
Decode Test 'sequence with special token' PASSED.
Decode Test 'empty list' PASSED.
Decode Test 'unknown token ID' PASSED.
Decode Test 'only special tokens' PASSED.
Decode Test 'unicode character from vocab' PASSED.

--- Testing encode/decode roundtrip ---
Roundtrip Test 0 for 'hello world!' PASSED.
Roundtrip Test 1 for '<EOS>hello<PAD> world€!' PASSED.
Roundtrip Test 2 for '复杂的中文字符串' PASSED.
Roundtrip Test 3 for '' PASSED.
Roundtrip Test 4 for '   leading and trailing spaces   ' PASSED.
Roundtrip Test 5 for ''s 'd 'll 've 're 'm 't' PASSED.
Roundtrip Test 6 for 'A single byte like z then a special <EOS> token.' PASSED.

--- Testing decode with malformed UTF-8 ---
Decode malformed UTF-8 PASSED. Decoded: ''h��i''

All Tokenizer class method tests (init, from_files, _encode_bytes, encode, encode_iterable, decode) completed.
